<a href="https://colab.research.google.com/github/shiling2007/Python-/blob/main/CVXPOT_mpc_1_1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://osqp.org/docs/examples/mpc.html

from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:100% !important; }</style>"))
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# from google.colab import drive
# drive.mount('/content/drive')
# from google.colab import files
# files.download('/content/drive/MyDrive/Colab Notebooks/Lease Payment Formula.ipynb') 
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.8f" % x))
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 999)
pd.set_option("max_colwidth", 500)
# try:
#  device_name = os.environ['COLAB_TPU_ADDR']
#  TPU_ADDRESS = 'grpc://' + device_name
#  print('Found TPU at: {}'.format(TPU_ADDRESS))
# except KeyError:
#  print('TPU not found')
%load_ext autoreload
%autoreload 2

In [13]:
# Generate data for control problem.
import numpy as np
from cvxopt import solvers
solvers.options['show_progress'] = True
import cvxpy as cp

np.random.seed(1)
n = 2
m = 1
T = 8
N = 10*T

In [14]:
A=np.array([[1,1],[0,1]])
B=np.array([[0],[1]])
x_0 = np.array([0,0])

In [15]:
A.shape, B.shape, A, B, x_0

((2, 2), (2, 1), array([[1, 1],
        [0, 1]]), array([[0],
        [1]]), array([0, 0]))

In [16]:
# Constraints
umin = np.array([-6]) 
umax = np.array([4]) 

In [17]:
# Define problem

x = cp.Variable((n, T+1))
u = cp.Variable((m, T))
x_init = cp.Parameter(n)

cost = 0

constr = [x[:,0] == x_init]
for t in range(T):
    cost += cp.sum_squares((x[1,t+1]-10)/10) + cp.sum_squares(u[:,t]) 

    constr += [x[:,t+1] == A@x[:,t] + B@u[:,t]]
    constr += [umin <= u[:,t], u[:,t] <= umax]

# sums problem objectives and concatenates constraints.
# constr += [x[1,T] == 10, x[:,0] == x_0]

problem = cp.Problem(cp.Minimize(cost), constr)


# Abstract— In this paper, we describe the embedded conic
# solver (ECOS), an interior-point solver for second-order cone
# programming (SOCP) designed specifically for embedded applications. ECOS is written in low footprint, single-threaded,
# library-free ANSI-C and so runs on most embedded platforms.
# The main interior-point algorithm is a standard primal-dual
# Mehrotra predictor-corrector method with Nesterov-Todd scaling and self-dual embedding, with search directions found via
# a symmetric indefinite KKT system, chosen to allow stable
# factorization with a fixed pivoting order. The indefinite system
# is solved using Davis’ SparseLDL package, which we modify
# by adding dynamic regularization and iterative refinement
# for stability and reliability, as is done in the CVXGEN code
# generation system, allowing us to avoid all numerical pivoting;
# the elimination ordering is found entirely symbolically. This
# keeps the solver simple, only 750 lines of code, with virtually no
# variation in run time. For small problems, ECOS is faster than
# most existing SOCP solvers; it is still competitive for mediumsized problems up to tens of thousands of variables.
# Disciplined convex programming (DCP) is a system for constructing mathematical expressions with known curvature from a given library of base functions
# ECOS interior point

In [18]:
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.8f" % x))
for nsim in range(N):
  x_init.value = x_0
  data, chain, inverse_data = problem.get_problem_data(solver=cp.ECOS)
  soln = chain.solve_via_data(problem, data)
  problem.unpack_results(soln, chain, inverse_data)
  x_key, u_key=list(problem.solution.primal_vars.keys())[0], list(problem.solution.primal_vars.keys())[1]
  x_0=problem.solution.primal_vars.get(x_key)[:,1]
  u_0=problem.solution.primal_vars.get(u_key)[:,0]
  print('nsim=', nsim, '\t'*2, 'x_0=', x_0, '\t',  'u_0=', u_0)


nsim= 0 		 x_0= [-0.00000000 0.64175074] 	 u_0= [0.64175074]
nsim= 1 		 x_0= [0.64175074 1.24231183] 	 u_0= [0.60056109]
nsim= 2 		 x_0= [1.88406258 1.80433535] 	 u_0= [0.56202352]
nsim= 3 		 x_0= [3.68839793 2.33029106] 	 u_0= [0.52595571]
nsim= 4 		 x_0= [6.01868899 2.82249380] 	 u_0= [0.49220274]
nsim= 5 		 x_0= [8.84118279 3.28310951] 	 u_0= [0.46061571]
nsim= 6 		 x_0= [12.12429230 3.71416516] 	 u_0= [0.43105565]
nsim= 7 		 x_0= [15.83845746 4.11755595] 	 u_0= [0.40339079]
nsim= 8 		 x_0= [19.95601341 4.49505973] 	 u_0= [0.37750378]
nsim= 9 		 x_0= [24.45107314 4.84833790] 	 u_0= [0.35327817]
nsim= 10 		 x_0= [29.29941105 5.17894796] 	 u_0= [0.33061005]
nsim= 11 		 x_0= [34.47835900 5.48833946] 	 u_0= [0.30939150]
nsim= 12 		 x_0= [39.96669846 5.77787348] 	 u_0= [0.28953402]
nsim= 13 		 x_0= [45.74457194 6.04882694] 	 u_0= [0.27095346]
nsim= 14 		 x_0= [51.79339889 6.30239161] 	 u_0= [0.25356467]
nsim= 15 		 x_0= [58.09579049 6.53968951] 	 u_0= [0.23729790]
nsim= 16 		 x_0= [64.63

In [ ]:
###end here

In [ ]:
print ("optimal value with ECOS:", problem.value)

optimal value with ECOS: 6.4174767412140685


In [ ]:
problem.solution.primal_vars

{4002: array([[-0.00000000, -0.00000000, 0.64175074, 1.83166651, 3.48164236, 5.50817762, 7.83153804, 10.37495791, 13.06387533],
        [-0.00000000, 0.64175074, 1.18991576, 1.64997585, 2.02653526, 2.32336042, 2.54341987, 2.68891742, 2.76130610]]),
 4003: array([[0.64175074, 0.54816502, 0.46006009, 0.37655941, 0.29682516, 0.22005945, 0.14549754, 0.07238868]])}

In [ ]:
x_key, u_key=list(problem.solution.primal_vars.keys())[0], list(problem.solution.primal_vars.keys())[1]
problem.solution.primal_vars.get(x_key),problem.solution.primal_vars.get(u_key)

(array([[-0.00000000, -0.00000000, 0.64175074, 1.83166651, 3.48164236, 5.50817762, 7.83153804, 10.37495791, 13.06387533],
        [-0.00000000, 0.64175074, 1.18991576, 1.64997585, 2.02653526, 2.32336042, 2.54341987, 2.68891742, 2.76130610]]),
 array([[0.64175074, 0.54816502, 0.46006009, 0.37655941, 0.29682516, 0.22005945, 0.14549754, 0.07238868]]))

In [ ]:
for t in range(T):
    cost += cp.sum_squares(x[:,t+1]) + cp.sum_squares(u[:,t])  \
        + cp.sum_squares( cp.exp( -0.5*(x[:,t+1] - xf[:,t+1]))) + cp.sum( cp.exp( -0.5*x[:,t+1] - xf[:,t+1]))  + cp.norm1(x[:,t+1]) + cp.tv(x[:,t+1]) \
         + cp.sum_squares(cp.abs(x[:,t+1] - xf[:,t+1] )) # +  cp.sum( exp( -(x[:,t+1]*x[:,t+1])))
    constr += [x[:,t+1] == A@x[:,t] + B@u[:,t],
               cp.norm(u[:,t], 'inf') <= 1]
# sums problem objectives and concatenates constraints.
constr += [x[:,T] == 0, x[:,0] == x_0]


problem = cp.Problem(cp.Minimize(cost), constr)
# problem.solve(solver=cp.ECOS, verbose=True)
%time problem.solve(solver=cp.ECOS, verbose=True)

# Disciplined convex programming (DCP) is a system for constructing mathematical expressions with known curvature from a given library of base functions
# ECOS interior point